# Feature Engineering: Presidential Contributions

Let's clean up the presidential contributions dataset by feature engineering.

In [1]:
# initialize Spark Session
import os
import sys
top_dir = os.path.abspath(os.path.join(os.getcwd(), "../"))
if top_dir not in sys.path:
    sys.path.append(top_dir)

from init_spark import init_spark
spark = init_spark()
spark

Initializing Spark...
Spark found in :  /Users/sujee/spark
Spark config:
	 spark.app.name=TestApp
	spark.master=local[*]
	executor.memory=2g
	spark.sql.warehouse.dir=/var/folders/lp/qm_skljd2hl4xtps5vw0tdgm0000gn/T/tmp8wckefoa
	some_property=some_value
Spark UI running on port 4040


In [2]:
%matplotlib inline
import time
from pyspark.sql.functions import isnan, when, count, col, split, trim, countDistinct, abs 
from pyspark.sql.types import IntegerType
import pyspark.sql.functions

## About the data
In US contributions to elections are public record published by Federal Election Commission (FEC).   
Here we have a sample of 10k for 2016 contributions.  

Optionally, To download the full dataset, you can execute the **download-data.sh**

## Step 1: Load the data

In [3]:
%%time
t1 = time.perf_counter()
#Load presidential contrib data
contribs = spark.read.csv("/data/presidential_election_contribs/2016/2016-100k.csv.gz",\
                          header=True, inferSchema=True)
t2 = time.perf_counter()
print("read {:,} records in {:,.2f} ms".format(contribs.count(), (t2-t1)*1000))



read 100,000 records in 3,785.15 ms
CPU times: user 3.71 ms, sys: 2.19 ms, total: 5.9 ms
Wall time: 4.73 s


In [16]:

## TODO: display all columns
contribs.printSchema()

## TODO : how many records do we have?
print ("count ", contribs.count())

## TODO : see some sample data
contribs.limit(5).toPandas()
#contribs.limit(5).toPandas().T

root
 |-- CMTE_ID: string (nullable = true)
 |-- CAND_ID: string (nullable = true)
 |-- CAND_NM: string (nullable = true)
 |-- CONTBR_NM: string (nullable = true)
 |-- CONTBR_CITY: string (nullable = true)
 |-- CONTBR_ST: string (nullable = true)
 |-- CONTBR_ZIP: string (nullable = true)
 |-- CONTBR_EMPLOYER: string (nullable = true)
 |-- CONTBR_OCCUPATION: string (nullable = true)
 |-- CONTB_RECEIPT_AMT: double (nullable = true)
 |-- CONTB_RECEIPT_DT: string (nullable = true)
 |-- RECEIPT_DESC: string (nullable = true)
 |-- MEMO_CD: string (nullable = true)
 |-- MEMO_TEXT: string (nullable = true)
 |-- FORM_TP: string (nullable = true)
 |-- FILE_NUM: integer (nullable = true)
 |-- TRAN_ID: string (nullable = true)
 |-- ELECTION_TP: string (nullable = true)

count  100000


,CMTE_ID,CAND_ID,CAND_NM,CONTBR_NM,CONTBR_CITY,CONTBR_ST,CONTBR_ZIP,CONTBR_EMPLOYER,CONTBR_OCCUPATION,CONTB_RECEIPT_AMT,CONTB_RECEIPT_DT,RECEIPT_DESC,MEMO_CD,MEMO_TEXT,FORM_TP,FILE_NUM,TRAN_ID,ELECTION_TP
0,C00605568,P20002671,"Johnson, Gary","SMITH, PAUL",SAN DIEGO,CA,92117,SELF,RETIRED,150.00,16-SEP-16,None,None,None,SA17A,1112281,SA17A.66792,G2016
1,C00574624,P60006111,"Cruz, Rafael Edward 'Ted'","BROWNE, THOMAS JOHN",WHITESBORO,NY,134921106,RETIRED,RETIRED,35.00,30-APR-15,None,None,None,SA17A,1024046,SA17.279055,P2016
2,C00580100,P80001571,"Trump, Donald J.","RISENHOOVER, LINDSEY",TULSA,OK,74133,INFORMATION REQUESTED,INFORMATION REQUESTED,73.59,09-NOV-16,None,X,None,SA18,1146165,SA18.136964,G2016
3,C00575795,P00003392,"Clinton, Hillary Rodham","RUIZ, GIULIANNA",NEW YORK,NY,100025651,TRILANTIC CAPITAL MANAGEMENT L.P.,CHIEF COMPLIANCE OFFICER & COUNSEL,500.00,19-SEP-16,None,X,* HILLARY VICTORY FUND,SA18,1137625,C11919741,G2016
4,C00575795,P00003392,"Clinton, Hillary Rodham","BRIGGS, AMY",BROOKLYN,NY,112314003,"SAP, INC",PARTNER,38.00,29-JUN-16,None,None,None,SA17A,1099613,C6305101,P2016


## Step 2 : Select fields to analyze


In [5]:
## TODO : extract the following fields
##    - Candidate Name : 'CAND_NM
##    - Contributor Name : 'CONTBR_NM'
##    - Contributor Occupation : ????
##    - COntribution Amount : ????
##    - contribution zip code : ???

contribs2 = contribs.select([ 'CAND_NM', 'CONTBR_NM', 'CONTBR_OCCUPATION', 'CONTBR_ZIP', 'CONTB_RECEIPT_AMT'])
print("count ", contribs2.count())
contribs2.show(20, False)

count  100000
+-------------------------+-------------------------+----------------------------------+----------+-----------------+
|CAND_NM                  |CONTBR_NM                |CONTBR_OCCUPATION                 |CONTBR_ZIP|CONTB_RECEIPT_AMT|
+-------------------------+-------------------------+----------------------------------+----------+-----------------+
|Johnson, Gary            |SMITH, PAUL              |RETIRED                           |92117     |150.0            |
|Cruz, Rafael Edward 'Ted'|BROWNE, THOMAS JOHN      |RETIRED                           |134921106 |35.0             |
|Trump, Donald J.         |RISENHOOVER, LINDSEY     |INFORMATION REQUESTED             |74133     |73.59            |
|Clinton, Hillary Rodham  |RUIZ, GIULIANNA          |CHIEF COMPLIANCE OFFICER & COUNSEL|100025651 |500.0            |
|Clinton, Hillary Rodham  |BRIGGS, AMY              |PARTNER                           |112314003 |38.0             |
|Clinton, Hillary Rodham  |BLAND, EDWARD  

## Step 3 : Register as SQL Tables

In [6]:
contribs.createOrReplaceTempView("contribs")
contribs2.createOrReplaceTempView("contribs2")

## list tables
spark.catalog.listTables()

[Table(name='contribs', database=None, description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='contribs2', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

## Step 4: Negative Contributions ??!??

As a cleanup, let us check for contribution amounts less than zero, and fix if necessary.

In [7]:
# Contribution receipt less than zero?
negative_contrib = contribs2.filter(' CONTB_RECEIPT_AMT < 0 ')

## TODO : how many negative contributions do we have?
print(negative_contrib.count())

## TODO : print some sample data
negative_contrib.show(10, False)

## SQL Query
spark.sql("select * from contribs2 where CONTB_RECEIPT_AMT < 0").show()

1388
+-------------------------+---------------------------+-----------------+----------+-----------------+
|CAND_NM                  |CONTBR_NM                  |CONTBR_OCCUPATION|CONTBR_ZIP|CONTB_RECEIPT_AMT|
+-------------------------+---------------------------+-----------------+----------+-----------------+
|Clinton, Hillary Rodham  |AZAM, MARIE                |null             |900682432 |-250.0           |
|Trump, Donald J.         |KEARNS, AIMEE              |RETIRED          |89149     |-78.05           |
|Carson, Benjamin S.      |AHTEN, HILDA               |null             |61554     |-500.0           |
|Clinton, Hillary Rodham  |CHARLES, MARTHE            |null             |330253825 |-50.0            |
|Clinton, Hillary Rodham  |LINDY, CATHY               |null             |341456344 |-500.0           |
|Cruz, Rafael Edward 'Ted'|HOLGUIN, CHENO             |ELECTRICIAN      |98662     |-59.0            |
|Clinton, Hillary Rodham  |HORWITZ, LAURA             |null         

## Step 5 : Extract data that only has positive contribution amount

In [8]:
## TODO : 
pos_contribs = contribs2.filter(' CONTB_RECEIPT_AMT > 0 ')

print(pos_contribs.count())

98608


In [9]:
## TODO : in SQL
pos_contribs2 = spark.sql("select * from contribs2 where CONTB_RECEIPT_AMT > 0 ")
pos_contribs2.show()

## now register this as a new table
pos_contribs2.createOrReplaceTempView("pos_contribs")

spark.sql("select count(*) from pos_contribs").show()

+--------------------+--------------------+--------------------+----------+-----------------+
|             CAND_NM|           CONTBR_NM|   CONTBR_OCCUPATION|CONTBR_ZIP|CONTB_RECEIPT_AMT|
+--------------------+--------------------+--------------------+----------+-----------------+
|       Johnson, Gary|         SMITH, PAUL|             RETIRED|     92117|            150.0|
|Cruz, Rafael Edwa...| BROWNE, THOMAS JOHN|             RETIRED| 134921106|             35.0|
|    Trump, Donald J.|RISENHOOVER, LINDSEY|INFORMATION REQUE...|     74133|            73.59|
|Clinton, Hillary ...|     RUIZ, GIULIANNA|CHIEF COMPLIANCE ...| 100025651|            500.0|
|Clinton, Hillary ...|         BRIGGS, AMY|             PARTNER| 112314003|             38.0|
|Clinton, Hillary ...|       BLAND, EDWARD|        MANUFACTURER| 280526251|             19.0|
|        Rubio, Marco|  SACK, ROGER D. MR.|             RETIRED| 922703416|            110.0|
|    Sanders, Bernard|CHURCH, MARY LOU LOU|             RETI

## Step 6: Split up name into first name and last name

In [10]:
split_col = split(contribs2['CONTBR_NM'], ',')
first_last_name = pos_contribs.withColumn('LASTNAME', trim(split_col.getItem(0)))
first_last_name = first_last_name.withColumn('FIRSTNAME', trim(split_col.getItem(1)))

first_last_name.show()

## register as sql table
first_last_name.createOrReplaceTempView("contribs3")

+--------------------+--------------------+--------------------+----------+-----------------+-----------+----------------+
|             CAND_NM|           CONTBR_NM|   CONTBR_OCCUPATION|CONTBR_ZIP|CONTB_RECEIPT_AMT|   LASTNAME|       FIRSTNAME|
+--------------------+--------------------+--------------------+----------+-----------------+-----------+----------------+
|       Johnson, Gary|         SMITH, PAUL|             RETIRED|     92117|            150.0|      SMITH|            PAUL|
|Cruz, Rafael Edwa...| BROWNE, THOMAS JOHN|             RETIRED| 134921106|             35.0|     BROWNE|     THOMAS JOHN|
|    Trump, Donald J.|RISENHOOVER, LINDSEY|INFORMATION REQUE...|     74133|            73.59|RISENHOOVER|         LINDSEY|
|Clinton, Hillary ...|     RUIZ, GIULIANNA|CHIEF COMPLIANCE ...| 100025651|            500.0|       RUIZ|       GIULIANNA|
|Clinton, Hillary ...|         BRIGGS, AMY|             PARTNER| 112314003|             38.0|     BRIGGS|             AMY|
|Clinton, Hillar

## Step 7: Analyze data
We can use Dataframe DSL language or SQL queries to perform analysis.  
Practice both.   
We will give you hints along the way.

### 7.1 - Count contributions per candidate

In [11]:
%%time

## TODO : What is the breakdown by candidate name?
## Hint : groupBy("name").count()

pos_contribs.groupBy("CAND_NM").count().show()

## SQL
s="""
select CAND_NM, count(*) as total_contribs, 
SUM(CONTB_RECEIPT_AMT) as total_contrib_amount
from pos_contribs
group by CAND_NM
order by total_contrib_amount DESC
"""

spark.sql(s).show()

+--------------------+-----+
|             CAND_NM|count|
+--------------------+-----+
|        Rubio, Marco| 1332|
|      Fiorina, Carly|  378|
|Christie, Christo...|   78|
|       Jindal, Bobby|   11|
|         Stein, Jill|  135|
|Gilmore, James S III|    3|
|    Sanders, Bernard|27427|
|      McMullin, Evan|   39|
|      Huckabee, Mike|   93|
|       Walker, Scott|   95|
|  Graham, Lindsey O.|   50|
|O'Malley, Martin ...|   86|
|     Kasich, John R.|  354|
|Santorum, Richard J.|   28|
| Carson, Benjamin S.| 3322|
|Webb, James Henry...|    6|
|    Lessig, Lawrence|   15|
|          Paul, Rand|  451|
|   Pataki, George E.|    5|
|       Johnson, Gary|  197|
+--------------------+-----+
only showing top 20 rows

+--------------------+--------------+--------------------+
|             CAND_NM|total_contribs|total_contrib_amount|
+--------------------+--------------+--------------------+
|Clinton, Hillary ...|         46622|   5797228.599999974|
|    Trump, Donald J.|         10213|  169

### 7.2 - Calculate AVG, MIN, MAX contributions per candidate
For this we can easily use SQL

In [12]:
%%time 

s="""
select CAND_NM, 
count(*) as total_contribs,
AVG(CONTB_RECEIPT_AMT)  as avg_contrib , 
MAX(CONTB_RECEIPT_AMT) as max_contrib, 
MIN(CONTB_RECEIPT_AMT) as min_contrib 
from pos_contribs
group by CAND_NM 
order by total_contribs desc, avg_contrib  DESC
"""


avg_per_candidate = spark.sql(s)
avg_per_candidate.show(20, False)

+-------------------------+--------------+------------------+-----------+-----------+
|CAND_NM                  |total_contribs|avg_contrib       |max_contrib|min_contrib|
+-------------------------+--------------+------------------+-----------+-----------+
|Clinton, Hillary Rodham  |46622         |124.34534340011096|5000.0     |0.15       |
|Sanders, Bernard         |27427         |47.18273599008275 |5000.0     |1.0        |
|Trump, Donald J.         |10213         |165.78047684323835|5400.0     |0.8        |
|Cruz, Rafael Edward 'Ted'|7278          |128.12899285517997|10800.0    |1.0        |
|Carson, Benjamin S.      |3322          |126.14475617098132|10000.0    |0.89       |
|Rubio, Marco             |1332          |462.09549549549547|10000.0    |3.0        |
|Paul, Rand               |451           |171.54108647450138|2700.0     |1.0        |
|Fiorina, Carly           |378           |236.98058201058203|3000.0     |3.0        |
|Bush, Jeb                |376           |1247.3693617

### 7.3 - Calculate AVG contribution by Occupation
If you are political consultants, which people you might try to solicit money?

In [13]:
## TODO 
#avg_per_occupation = spark.sql("")

### 7.4 - Find Zipcodes that give more money

In [14]:
%%time 

s="""
select  substring(CONTBR_ZIP, 0, 5) as zip, AVG(CONTB_RECEIPT_AMT) as avg_contribs, COUNT(*) as cnt  
from pos_contribs 
group by zip 
-- having cnt > 10
order by avg_contribs DESC
"""

spark.sql(s).show()

+-----+------------------+---+
|  zip|      avg_contribs|cnt|
+-----+------------------+---+
|29641|            5400.0|  1|
|84133|            5400.0|  1|
|32340|            5400.0|  1|
|80544|            5000.0|  1|
|76437|2783.3333333333335|  3|
|21057|2733.3333333333335|  3|
|98464|            2725.0|  2|
|36502|            2712.5|  2|
|75935|2708.3333333333335|  3|
|45883|            2700.0|  1|
|79901|            2700.0|  1|
|91522|            2700.0|  1|
|60075|            2700.0|  1|
|23218|            2700.0|  1|
|22106|            2700.0|  1|
|48120|            2700.0|  1|
|27114|            2700.0|  1|
|34266|            2700.0|  1|
|07417|            2700.0|  1|
|15331|            2700.0|  1|
+-----+------------------+---+
only showing top 20 rows

CPU times: user 1.22 ms, sys: 1.15 ms, total: 2.37 ms
Wall time: 1.32 s


## Class Discussion
Now that we have done some feature engineering, waht other attributes we can extract and analyze?